In [38]:
using CSV
using Glob
using DataFrames
using Missings
using ProgressBars
using ArchGDAL

In [39]:
# confirm that sensor numbers/locations did not change year-to-year
sensor_meta = vcat(map(glob("../data/d*meta*.tsv")) do fn
            return CSV.read(fn, DataFrame, delim="\t")
    end...)
nothing

┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 970. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 974. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 1265. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 1266. Ignoring any extra columns on this row
└ @ CSV /Users/matthewc/.julia/packages/CSV/Zl2ww/src/file.jl:613
┌ Warning: thread = 2 warning: parsed expected 18 columns, but didn't reach end of line around data row: 1369. Ignoring any extra columns 

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude
,Int64,Int64,String,Int64,Int64,Int64?,Any,Float64,Float64?,Float64?
1,308511,50,E,3,17,missing,31.627,60.162,38.7611,-120.57
2,308512,50,W,3,17,missing,31.627,60.166,38.7612,-120.57
3,311831,5,S,3,67,missing,10.896,506.189,38.4098,-121.484
4,311832,5,S,3,67,missing,10.896,506.189,38.4098,-121.484
5,311844,5,N,3,67,missing,11.105,506.398,38.4128,-121.484
6,311847,5,N,3,67,missing,12.185,507.478,38.4283,-121.488
7,311864,5,N,3,67,missing,11.933,507.226,38.4246,-121.487
8,311903,50,E,3,67,64000,L0.633,3.789,38.5669,-121.506
9,311930,50,E,3,67,64000,L0.632,3.788,38.5669,-121.506


In [45]:
uzas = DataFrame(ArchGDAL.getlayer(ArchGDAL.read("../data/tl_2020_us_uac10.shp"), 0))
nothing

In [47]:
println(join(sort(uzas.NAME10[occursin.(["CA"], uzas.NAME10)]), "\n"))

Alturas, CA
Angels, CA
Antioch, CA
Arbuckle, CA
Arcata--McKinleyville, CA
Arroyo Grande--Grover Beach, CA
Arvin, CA
Auburn--North Auburn, CA
Avalon, CA
Avenal, CA
Bakersfield, CA
Barstow, CA
Big Bear City, CA
Bishop, CA
Blythe, CA--AZ
Bonadelle Ranchos-Madera Ranchos, CA
Brawley, CA
Burney, CA
California City, CA
Calipatria, CA
Calistoga, CA
Camarillo, CA
Cambria, CA
Carmel Valley Village, CA
Chico, CA
Chowchilla, CA
Clearlake Riviera, CA
Clearlake, CA
Cloverdale, CA
Coalinga, CA
Colfax, CA
Colusa, CA
Concord, CA
Corcoran, CA
Corning, CA
Cottonwood, CA
Crescent City, CA
Davis, CA
Delano, CA
Desert Hot Springs, CA
Discovery Bay, CA
Dixon, CA
Dos Palos, CA
Earlimart, CA
East Quincy, CA
El Centro--Calexico, CA
El Paso de Robles (Paso Robles)--Atascadero, CA
Escalon, CA
Esparto, CA
Eureka, CA
Fairfield, CA
Fillmore, CA
Firebaugh, CA
Forestville, CA
Fort Bragg, CA
Fort Irwin, CA
Fortuna, CA
Frazier Park, CA
Fresno, CA
Galt, CA
Gilroy--Morgan Hill, CA
Gonzales, CA
Grass Valley, CA
Greenfield

In [54]:
sensor_meta.geom = passmissing(ArchGDAL.createpoint).(sensor_meta.Longitude, sensor_meta.Latitude)
nothing

In [55]:
# filter to just California
ca_uza_geom = uzas[occursin.(["CA"], uzas.NAME10), ""]
nothing

In [63]:
sensor_meta.urban = map(sensor_meta.geom) do g
    if ismissing(g)
        return missing
    else
        for poly in ca_uza_geom
            if ArchGDAL.contains(poly, g)
                return true
            end
        end
    end
    
    return false  # not in any UZA
end
nothing

In [65]:
sum(skipmissing(sensor_meta.urban))

16218

In [67]:
sum(.!skipmissing(sensor_meta.urban))

2430

In [66]:
CSV.write("../data/sensor_meta_geo.csv", sensor_meta)

"../data/sensor_meta_geo.csv"